In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import os
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction import text 
from sklearn.decomposition import LatentDirichletAllocation as LDA
from pyLDAvis import sklearn as sklearn_lda
import pyLDAvis

# import lda


C:\Users\jjing\Anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\Users\jjing\Anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


In [2]:
# max rows and columns
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)
# display all contents in dataframe
pd.option_context("display.max_colwidth", -1, "display.max_rows", None)

In [3]:
df = pd.DataFrame()
for file in os.listdir("../data/"):
    df = pd.concat([df, pd.read_csv("../data/"+file)], axis=0)

In [4]:
df.isnull().sum()

subreddit        0
created_utc      0
body           754
score            0
dtype: int64

In [5]:
df = df.dropna()

In [6]:
df['year'] = [datetime.fromtimestamp(utc).year for utc in df['created_utc']]
df['month'] = [datetime.fromtimestamp(utc).month for utc in df['created_utc']]

In [7]:
df['year_month'] = df['year'].map(str) + '-' + df['month'].map(str)
df['year_month'] = pd.to_datetime(df['year_month'], format='%Y-%m').dt.strftime('%Y-%m')

In [8]:
df.drop(columns=['created_utc'], axis=1, inplace=True)

In [9]:
df['subreddit'].unique()

array(['HeadphoneAdvice', 'headphones'], dtype=object)

In [10]:
df[df['subreddit']!='HeadphoneAdvice']['year'].unique()

array([2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010],
      dtype=int64)

In [11]:
df.head()

,subreddit,body,score,year,month,year_month
0,HeadphoneAdvice,[PA] Upgrade from HyperX cloud 2's.I have been...,1,2020,2,2020-02
1,HeadphoneAdvice,"Neutral, USB stick style DAC, $100-200 range. ...",1,2020,2,2020-02
2,HeadphoneAdvice,Sennheiser HD 4.5 BTNC vs JBL Live 650BTNCLook...,1,2020,2,2020-02
3,HeadphoneAdvice,Best General Use IEMS under 200?I've been look...,1,2020,2,2020-02
4,HeadphoneAdvice,What gaming headphones should I buy?I have use...,1,2020,2,2020-02


In [12]:
stopwords_final = text.ENGLISH_STOP_WORDS.union({'https', 'http', 'www', 'com', 'headphones', 'headphone', 'earphones', 'earphone'})

In [13]:
cvec = CountVectorizer(ngram_range=(1,2), min_df=5, max_df=0.9, max_features=20000, stop_words=stopwords_final)
X = cvec.fit_transform(df['body'])

In [14]:
n_topics = 20
# Create and fit the LDA model
lda_model = LDA(n_components=n_topics, n_jobs=4)
lda_model.fit(X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=4,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [15]:
top_words = 20
for i, topic in enumerate(lda_model.components_):
    words = np.array(cvec.get_feature_names())[np.argsort(topic)][:-(top_words+1):-1]
    print('Topic {}: {}'.format(i, '_'.join(words)))

Topic 0: just_like_ve_got_don_know_really_time_pair_right_think_new_people_bought_try_way_did_ago_didn_ll
Topic 1: amp_amazon_dp_ref_sr_utf8_product_utf8 amp_qid_amp sr_amp qid_keywords_th_uk_amp keywords_gp_x00_psc_amp psc_gp product
Topic 2: deleted_post_review_impressions_share_flair_need_feel_message_album_rig_harman_comment_interesting_did_image_discussion_target_impressions review_spotify
Topic 3: amp_amp x200b_x200b_audio_volume_sound_use_phone_using_usb_just_music_pc_jack_work_quality_does_plug_computer_way
Topic 4: gaming_headset_mic_sound_good_pc_looking_need_use_like_games_headsets_cloud_game_quality_microphone_want_better_hyperx_music
Topic 5: iems_iem_ve_good_thanks_kz_audio_pro_price_t2_tin_like_better_heard_reviews_audeze_best_lcd_look_ll
Topic 6: pads_dt_pro_ohm_beyerdynamic_akg_990_250_dt990_dt770_earpads_dt 990_ohms_impedance_80_beyerdynamic dt_koss_770_ear pads_db
Topic 7: looking_ear_pair_good_earbuds_wireless_like_quality_need_want_don_help_thanks_budget_know_reall

In [16]:
LDAvis_prepared = sklearn_lda.prepare(lda_model, X, cvec)
pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(n_topics) +'.html')

C:\Users\jjing\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [17]:
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.128769  0.105333       1        1  13.454359
19     0.136581  0.123207       2        1  10.867524
7      0.097843 -0.062139       3        1   8.561497
11     0.096188 -0.043277       4        1   7.083938
15     0.153936  0.048459       5        1   7.040974
3      0.093204  0.085615       6        1   6.709881
4      0.097398 -0.088217       7        1   5.729021
12     0.104481  0.015631       8        1   4.967515
10     0.040930 -0.089085       9        1   4.212332
18    -0.049321 -0.130432      10        1   4.104860
5      0.064522  0.000080      11        1   4.019520
14     0.024386  0.152659      12        1   3.984737
17     0.085413  0.075079      13        1   3.564321
16     0.010241 -0.206329      14        1   3.557904
6      0.011215 -0.030549      15        1   3.021688
9     -0.367236 -0.033845      16        1   2.6809

In [14]:
lda_X = lda.LDA(n_topics=20, random_state=42)
lda_X.fit(X)

INFO:lda:n_documents: 372008
INFO:lda:vocab_size: 20000
INFO:lda:n_words: 15007010
INFO:lda:n_topics: 20
INFO:lda:n_iter: 2000
C:\Users\jjing\Anaconda3\lib\site-packages\lda\utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -180140394
INFO:lda:<10> log likelihood: -142531918
INFO:lda:<20> log likelihood: -131203715
INFO:lda:<30> log likelihood: -128239952
INFO:lda:<40> log likelihood: -127109346
INFO:lda:<50> log likelihood: -126549389
INFO:lda:<60> log likelihood: -126215583
INFO:lda:<70> log likelihood: -125993341
INFO:lda:<80> log likelihood: -125818127
INFO:lda:<90> log likelihood: -125706881
INFO:lda:<100> log likelihood: -125617914
INFO:lda:<110> log likelihood: -125568552
INFO:lda:<120> log likelihood: -125518962
INFO:lda:<130> log likelihood: -125466264
IN

INFO:lda:<1830> log likelihood: -125095914
INFO:lda:<1840> log likelihood: -125094561
INFO:lda:<1850> log likelihood: -125102810
INFO:lda:<1860> log likelihood: -125099965
INFO:lda:<1870> log likelihood: -125093485
INFO:lda:<1880> log likelihood: -125091465
INFO:lda:<1890> log likelihood: -125093860
INFO:lda:<1900> log likelihood: -125100967
INFO:lda:<1910> log likelihood: -125095476
INFO:lda:<1920> log likelihood: -125094491
INFO:lda:<1930> log likelihood: -125097785
INFO:lda:<1940> log likelihood: -125092070
INFO:lda:<1950> log likelihood: -125093284
INFO:lda:<1960> log likelihood: -125095979
INFO:lda:<1970> log likelihood: -125095766
INFO:lda:<1980> log likelihood: -125094820
INFO:lda:<1990> log likelihood: -125100728
INFO:lda:<1999> log likelihood: -125096301


In [36]:
top_words = 20
for i, topic in enumerate(lda_X.topic_word_):
    words = np.array(cvec.get_feature_names())[np.argsort(topic)][:-(top_words+1):-1]
    print('Topic {}: {}'.format(i, '_'.join(words)))


Topic 0: com_https_www_https www_http_amp_imgur_headphones_imgur com_youtube_watch_youtube com_www youtube_com watch_jpg_reddit_headphone_review_http www_http imgur
Topic 1: wireless_earbuds_noise_headphones_bluetooth_ear_sound_quality_good_looking_cancelling_noise cancelling_wired_sound quality_use_best_like_earphones_wireless headphones_don
Topic 2: headphones_ath_audio_open_technica_audio technica_m50x_closed_sound_technica ath_better_beats_pair_m40x_good_removed_akg_m50_ath m50x_best
Topic 3: gaming_headset_sound_mic_headphones_good_pc_music_use_games_audio_cloud_quality_game_headsets_need_like_better_hyperx_using
Topic 4: headphones_volume_sound_high_like_hear_low_just_don_headphone_loud_difference_impedance_music_people_make_different_does_noise_listening
Topic 5: bass_sound_like_treble_good_really_soundstage_bit_just_better_mids_great_headphone_sounds_signature_little_highs_think_feel_ve
Topic 6: ear_pads_ears_like_comfortable_head_just_headphones_don_fit_ve_comfort_headband_use

In [32]:
np.array(cvec.get_feature_names())

array(['00', '00 filter', '000', ..., 'zx', 'zx300', 'zxr'], dtype='<U110')

In [41]:
lda_X.doc_topic_

array([[0.00128205, 0.00128205, 0.00128205, ..., 0.15512821, 0.00128205,
        0.00128205],
       [0.00227273, 0.00227273, 0.00227273, ..., 0.00227273, 0.18409091,
        0.00227273],
       [0.00333333, 0.23666667, 0.00333333, ..., 0.17      , 0.00333333,
        0.03666667],
       ...,
       [0.025     , 0.025     , 0.025     , ..., 0.025     , 0.025     ,
        0.025     ],
       [0.00294118, 0.00294118, 0.00294118, ..., 0.00294118, 0.00294118,
        0.00294118],
       [0.00769231, 0.08461538, 0.00769231, ..., 0.00769231, 0.00769231,
        0.00769231]])

In [42]:
lda_X.components_

array([[3.47737931e-04, 2.25789190e-08, 2.14522309e-04, ...,
        2.25789190e-08, 2.25789190e-08, 2.25789190e-08],
       [1.03788164e-08, 1.03788164e-08, 1.03788164e-08, ...,
        1.03788164e-08, 1.03788164e-08, 1.03788164e-08],
       [1.12893061e-04, 1.88123748e-08, 1.88123748e-08, ...,
        1.88123748e-08, 1.88123748e-08, 1.88123748e-08],
       ...,
       [1.00094964e-04, 2.12922705e-08, 2.12922705e-08, ...,
        2.12922705e-08, 2.12922705e-08, 2.12922705e-08],
       [1.86523317e-08, 1.86523317e-08, 1.06336943e-04, ...,
        1.86523317e-08, 2.10790001e-04, 1.86523317e-08],
       [3.00628916e-08, 3.00628916e-08, 3.00628916e-08, ...,
        1.32306786e-04, 1.59363388e-04, 3.00628916e-08]])

In [25]:
tvec = TfidfVectorizer(ngram_range=(1,2), min_df=5, max_df=0.9, max_features=20000, stop_words='english')
X_train_tvec = tvec.fit_transform(X_train)
X_train_df = pd.DataFrame(X_train_tvec.toarray(), columns=tvec.get_feature_names())
X_train_df.head()

,00,00 filter,000,000 hz,003,007,009,009s,01,02,02 headphone,03,04,05,06,07,08,09,0a,0aver,0aver whtgcfjdepldtbwsvfleg5px3zocnfdvxdodve,0db,0s,10,10 10,10 15,10 20,10 best,10 db,10 headphones,10 hours,10 minutes,10 output,10 pc,10 seconds,10 sound,10 times,10 year,10 years,100,100 120,100 130,100 150,100 200,100 250,100 300,100 amazon,100 amp,100 better,100 bucks,100 budget,100 cad,100 dac,100 dollar,100 dollars,100 don,100 euro,100 euros,100 good,100 headphone,100 headphones,100 just,100 like,100 looking,100 max,100 pair,100 preferably,100 price,100 range,100 really,100 removed,100 source,100 sure,100 thanks,100 usd,100 use,100 ve,100 volume,100 want,100 willing,100 worth,1000,1000 headphones,1000 xm3,1000mx2,1000mx3,1000x,1000x m3,1000xm2,1000xm3,1000xm3 bose,1000xm3 vs,1000xm3s,100hz,100i,100ish,100s,100usd,101,102,104,105,1060,108,109,10db,10ft,10k,10khz,10s,10th,10x,11,11 11,11 28,110,1100,110db,115,119,12,12 clock,12 hours,120,1200,125,1266,128,129,13,130,135,135 recommendations,1350,139,14,140,145,149,15,15 20,15 minutes,15 years,150,150 200,150 250,150 300,150 amazon,150 budget,150 cad,150 dollar,150 dollars,150 ear,150 headphones,150 looking,150 max,150 ohm,150 ohms,150 price,150 range,150 removed,150 source,150 thanks,150 usd,1500,150i,150ohm,150usd,151b0abg1uzh2sfik,151b0abg1uzh2sfik mqpkik2jd4khqpxlfydisewvoq,1540,155,159,16,16 44,16 bit,16 ohm,16 ohms,160,1600,165,169,16bit,17,170,175,1770,1770 pro,177x,179,18,180,185,19,190,192,192khz,195,1964,199,1990,1990 pro,1990 pros,1990s,1a,1am2,1db,1k,1khz,1more,1more quad,1more stylish,1more triple,1r,1rl22ot5olcfrjzmhxigv89zped1bbj2p6zxcwwry0gy,1rl22ot5olcfrjzmhxigv89zped1bbj2p6zxcwwry0gy viewform,1s,1st,1st gen,1x,20,20 30,20 40,20 amp,20 bucks,20 dollars,20 hours,20 hz,20 minutes,20 years,200,200 250,200 300,200 400,200 500,200 amp,200 aud,200 bucks,200 budget,200 cad,200 dollar,200 dollars,200 euro,200 euros,200 good,200 headphones,200 like,200 looking,200 max,200 pair,200 price,200 range,200 really,200 removed,200 source,200 thanks,200 usd,200 ve,200 willing,2000,2000s,2009,200i,200ish,200usd,201,2010,2011,2011 02,2012,2013,2013 sony,2014,2015,2016,2017,2018,2018 https,2019,202,2020,205,206,20error,20hz,20hz 20khz,20khz,20removal,20removal 20error,20s,21,210,215,215s,22,220,225,23,230,24,24 96,24 bit,24 hours,240,249,24bit,25,25 ii,250,250 300,250 600,250 ohm,250 ohms,250 source,250 usd,2500,250ohm,250ohms,250ω,256,256112744011530242,256112744011530242 285358055514701835,256112744011530242 304585848559632385,256112744011530242 363963783728857090,25s,26,260,27,270,275,28,280,280 pro,280 pros,280s,285358055514701835,285358055514701835 download,29,29 pdf,290,299,29e146,29e146 common_headphone_questions_answered,2c,2caps,2cs,2db,2f,2f 2fwww,2fwww,2i,2i2,2i4,2k,2khz,2m,2nd,2nd gen,2nd generation,2nd hand,2pin,2s,2se,2u,2v,2x,30,30 40,30 50,30 bucks,30 day,30 days,30 dollars,30 headphones,30 hours,30 http,30 minutes,30 seconds,300,300 350,300 400,300 500,300 budget,300 cad,300 dollar,300 dollars,300 headphones,300 ii,300 ohm,300 ohms,300 pair,300 price,300 range,300 removed,300 source,300 usd,3000,300ohm,300ohms,300s,300usd,304585848559632385,304585848559632385 snoo_neu,31,3100,315,32,32 80,32 bit,32 ohm,32 ohms,320,320 kbps,320kbps,320kbps mp3,325,32bit,32ohm,32ohms,33,330,336se,34,349,35,35 ii,35 word,350,350 usd,35ii,35mm,35s,36,360,363963783728857090,363963783728857090 snoo_neu,363d,37,370,373d,38,380,380 pro,39,399,3a,3a 2f,3a weekly,3d,3d audio,3d printed,3d sound,3db,3k,3m,3mm,3rd,3rd gen,3rd party,3s,3vt0mn,3vt0mn rheadphones_is_creating_an_audiophile_playlist,3x,40,40 50,40 60,40 bt,40 bucks,400,400 500,400 budget,400 headphones,400 usd,4000,400i,400s,404,40bt,40mm,40s,40x,41,42,425,429,43,439,44,44 1khz,440,449,45,450,46,47,48,48 hours,480,48khz,...,wisdom,wise,wish,witch,witcher,withstand,wm1a,woke,woman,won,won able,won break,won fall,won fit,won hear,won make,won need,won really,won sound,won turn,won use,won using,won work,wonder,wondered,wonderful,wonderfull